In [125]:
import numpy as np
import json
import pandas as pd

In [126]:
data = pd.read_csv("../bounding_box_data.csv", sep=",")
data.insert(1, "filename", data["timestamp"] + "_infrared_image.png")
data.insert(2, "class", "Weigher")

column_names = ["deviceID", "filename", "class", "timestamp", "x", "y", "width", "height"]
json_out_path = "yolov5/coco_annotation.json"

images = []
categories = []
annotations = []

data.head(5)

,deviceID,filename,class,timestamp,x,y,width,height
0,0a:92:5e:29:ec:a2,20191225T000100+0900_infrared_image.png,Weigher,20191225T000100+0900,660,388,125,125
1,0a:92:5e:29:ec:a2,20191225T000606+0900_infrared_image.png,Weigher,20191225T000606+0900,660,388,125,125
2,0a:92:5e:29:ec:a2,20191225T001111+0900_infrared_image.png,Weigher,20191225T001111+0900,660,388,125,125
3,0a:92:5e:29:ec:a2,20191225T001617+0900_infrared_image.png,Weigher,20191225T001617+0900,660,388,125,125
4,0a:92:5e:29:ec:a2,20191225T002120+0900_infrared_image.png,Weigher,20191225T002120+0900,660,388,125,125


In [127]:
info = {
        "year": "2021",
        "version": "1",
        "description": "Created by Rune Wind",
        "contributor": "Rune Wind",
        "url": "https://public.roboflow.ai/object-detection/undefined",
        "date_created": "2021-10-19T12:16:31+00:00"
    }

licenses = [
        {
            "id": 1,
            "url": "https://creativecommons.org/licenses/by/4.0/",
            "name": "CC BY 4.0"
        }
    ]

categories = [
        {
            "id": 0,
            "name": "Weighers",
            "supercategory": "none"
        },
        {
            "id": 1,
            "name": "Weigher",
            "supercategory": "Weighers"
        }]

In [128]:
# category = {}
# category["supercategory"] = "none"
# category["id"] = 0
# category["name"] = "Weigher"
# categories.append(category)

In [129]:
data['fileid'] = data['filename'].astype('category').cat.codes
data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
data['categoryid'] = data['categoryid']+1
data['annid'] = data.index

In [130]:
def image(row):
    image = {}
    image["height"] = 720
    image["width"] = 1280
    image["id"] = row.fileid
    image["file_name"] = row.filename
    return image

In [131]:
# def category(row):
#     category = {}
#     category["supercategory"] = 'None'
#     category["id"] = row.categoryid
#     category["name"] = row[2]
#     return category

In [132]:
def annotation(row):
    annotation = {}
    area = row.width*row.height
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid

    annotation["bbox"] = [row.x, row.y, row.width, row.height]

    annotation["category_id"] = 1
    annotation["id"] = row.annid
    return annotation

In [133]:
for row in data.itertuples():
    annotations.append(annotation(row))

imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
for row in imagedf.itertuples():
    images.append(image(row))

# catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
# for row in catdf.itertuples():
#     categories.append(category(row))

In [134]:
data_coco = {}
data_coco["info"] = info
data_coco["licenses"] = licenses
data_coco["categories"] = categories
data_coco["images"] = images
data_coco["annotations"] = annotations
json.dump(data_coco, open(json_out_path, "w+"), indent=4)